# Exploring Lexington's Redlined Neighborhoods: Wealth

This notebook covers the steps taken to collect and pre-process data for redlining-wealth-health-lexington github repo.

A fair amount of data for Lexington can be found in their archub data portal
https://data-lfucg.hub.arcgis.com/

## Education Aspect

- Education location proximity of institutions or libraries through time (as higher education can lead to higher income)

    - [Current Data](https://data-lfucg.hub.arcgis.com/datasets/67c201f27cd24921a21c69a420f6cae1_0/explore?location=38.039783%2C-84.489022%2C12.21). 

    - Historical Data?

- Highschool education/college education (US census bureau)
- Computer/Internet Accessibility
- Median Household income
- Mortgage approval rates through time (HMDA home mortgage disclosure act data)
- Percent homeownership today (US census bureau)
- Property values today (county clerks office?)
- Segregation
- Income
- Poverty
- Race 

### US Census bureau data 
[Video](https://www.youtube.com/watch?v=1LZPYS0cR68) detailing how to access data for specific census tracts
Can find US Census data [here](https://mtgis-portal.geo.census.gov/arcgis/apps/MapSeries/index.html?appid=2566121a73de463995ed2b2fd7ff6eb7) on the 2020 Census Bureau Demographic Data Map Viewer OR [here](data.census.gov)

Data within the US Census bureau is organized in a hierarchy of:
Nation-> State -> County -> Tract -> Block Group -> Block
Where a Nation is composed of my states which are composed of many counties etc..

Will need to use these sites to determine the exact census tracts for data interested in. Possible this will be how to perform table joins.

data.census.gov
From here click advanced search and select what filters to work through.
Geography example:
Click advanced search-geography filter-select the hierarchy level wish to pull-state-county-within (tract)
To only pull data from 2020 census click on year filter and select year of interest
To only pull data from specific census click on surveys filter and select survey of interest
Click search (data tables resulting from these filters will populate)
Can then either download the table in csv or look in map format

For example I chose
advanced search-geography-block-kentucky-fayette-all blocks within fayette county

Promising survey:
**DP4 Profile of Selected Housing Characteristics: 2000** Decennial Survey
**DP04 Selected Housing Characteristics (2019: ACS American Community Survey 5 year estimates data profiles** 
Housing units, Structure type, year built, number of rooms, year householder moved into unit (establish long term owners), vehicles, house heating, specified owner occupied units value..., gross rent, 

**DP02 Selected Social Characteristics in the United States (2019: ACS 5-Year Estimates Data Profiles**
Educational attainment, Disability status, residence length, with a computer, with broadband internet


To Do:
*Determine what years data is available for interest, then select correct census tract that best corresponds to these
*Open the home mortgage disclosure act to explore what attributes are within. 
*Download other data sets of interest and determin what attributes are within. (use jupyter packages-will need to download packages into the envrionment first)
*Determine how many parcels are within the tracts and contact fayette county clerks office
*Overlay census tracts with redlining geoJSON to determine what tracts to pull



